In [4]:
###################################################################
# Application of Lucas-Kanade optical flow method on a video
###################################################################

import cv2
import numpy as np
import os

path = os.getcwd()

# Path for saving the optical flow video
result_path = os.path.join(path,'Results')

# Parameters for Shi-Tomasi corner detection
feature_params = dict(maxCorners = 30, qualityLevel = 0.2, minDistance = 2, blockSize = 7)

# Parameters for Lucas-Kanade optical flow
lk_params = dict(winSize = (15,15), maxLevel = 3, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Variable for green color to draw optical flow track
color = (0, 255, 0)

vid = cv2.VideoCapture('Videos/video.mp4')
ret, first_frame = vid.read()
prev_frame_gray = cv2.cvtColor(first_frame,cv2.COLOR_BGR2GRAY)
good_features = cv2.goodFeaturesToTrack(prev_frame_gray, mask = None, **feature_params)
mask = np.zeros_like(first_frame)

# Size of the video frame
frame_width = int(vid.get(3))
frame_height = int(vid.get(4))
frame_size = (frame_width, frame_height)
# Frame rate of video
frame_rate = 30

result = cv2.VideoWriter(result_path+'/Result.avi', cv2.VideoWriter_fourcc(*'MJPG'),frame_rate, frame_size)

while(True):
    
    ret, next_frame = vid.read()
    if ret == False:
        break
    next_img_gray = cv2.cvtColor(next_frame,cv2.COLOR_BGR2GRAY)
    # Apply Lucas-Kanade method 
    next_pts, status, error = cv2.calcOpticalFlowPyrLK(prev_frame_gray, next_img_gray, good_features, None, **lk_params)
    
    good_features_old = good_features[status == 1]
    good_features_new = next_pts[status == 1]
    
    # To draw the optical flow tracks
    for i, (new_features, old_features) in enumerate(zip(good_features_new, good_features_old)):
        # x,y coordinates of the feature points in the new frame
        a, b = new_features.ravel()
        # x,y coordinates of the feature points in the old frame
        c, d = old_features.ravel()
        # Drawing a line to track the pixels
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color, 2)
        # Draw a circle around the tracked pixel in the new frame
        next_frame = cv2.circle(next_frame, (int(a), int(b)), 3, color, -1)
        
    # Overlaying the optical flow tracks on the original frame
    output_img = cv2.add(next_frame, mask)
    # Assign the current frame to the previous frame which is used for the next iteration
    prev_frame_gray = next_img_gray.copy()
    # Assign the features detected in the current frame to the features of the previous frame
    good_features = good_features_new.reshape(-1, 1, 2)
    
    cv2.imshow('sparse optical flow (Press q to exit)',output_img)
    
    # To save the video (Uncomment the line to save)
    #result.write(output_img)
    
    if cv2.waitKey(frame_rate) & 0xFF == ord('q'):
        break
        
vid.release()
result.release()
cv2.destroyAllWindows()